In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime, date

#### Feed the downloaded file here

In [2]:
df = pd.read_csv(r'C:\Users\user\Downloads\transactions2.csv')

In [3]:
df.columns = df.columns.str.strip()
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2022-08-26,5102363071,Quant Small Cap Growth Direct Plan,buy,36.491,137.0145,137.0145,4999.75
1,2022-08-18,5102363071,Quant Mid Cap Growth Direct Plan,buy,36.697,136.2458,138.0588,4999.75
2,2022-08-16,5102363071,Quant Infrastructure Growth Direct Plan,buy,213.192,23.4519,23.7855,4999.75
3,2022-08-11,5102363071,Quant Active Growth Direct Plan,buy,22.671,441.0636,450.8031,9999.50
4,2022-08-11,5102363071,Quant Infrastructure Growth Direct Plan,buy,432.598,23.1150,23.7855,9999.50


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
df2 = df

#### Prevent all the liquid fund from showing

In [6]:
df = df[~df['Name of the Fund'].str.contains("Kuvera|Liquid|Money|Gilt|Cash|Bond|Debt|Short|Gold|Securities|Duration|Income")]
##remove the upper line in case performance of all the mutual fund is required
df = df.loc[~df['Order'].str.contains("sell")]

In [7]:
today = pd.to_datetime(date.today())

#### Control the number of days to look behind / calculate the Xirr for. for three years we take 365*3 = 1095 days

In [8]:
df = df.loc[df['Date'] > (pd.to_datetime(date.today() - timedelta(days = 400)))]

In [9]:
mf_name = df['Name of the Fund'].unique()

#### See the list of all the mutual funds

In [10]:
mf_name

array(['Quant Small Cap Growth Direct Plan',
       'Quant Mid Cap Growth Direct Plan',
       'Quant Infrastructure Growth Direct Plan',
       'Quant Active Growth Direct Plan',
       'PGIM India Midcap Opportunities Growth Direct Plan',
       'Canara Robeco Small Cap Growth Direct Plan',
       'Quant Tax Growth Direct Plan',
       'Mirae Asset Tax Saver Growth Direct Plan',
       'Axis Small Cap Growth Direct Plan',
       'Mirae Asset Emerging Bluechip Growth Direct Plan',
       'ICICI Prudential Commodities Growth Direct Plan',
       'Tata Digital India Growth Direct Plan',
       'ICICI Prudential Technology Growth Direct Plan',
       'ICICI Prudential Infrastructure Growth Direct Plan',
       'PGIM India Flexi Cap Growth Direct Plan',
       'Axis Growth Opportunities Growth Direct Plan'], dtype=object)

In [11]:
df['Amount (INR)'] = df['Amount (INR)']*-1

In [12]:
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2022-08-26,5102363071,Quant Small Cap Growth Direct Plan,buy,36.491,137.0145,137.0145,-4999.75
1,2022-08-18,5102363071,Quant Mid Cap Growth Direct Plan,buy,36.697,136.2458,138.0588,-4999.75
2,2022-08-16,5102363071,Quant Infrastructure Growth Direct Plan,buy,213.192,23.4519,23.7855,-4999.75
3,2022-08-11,5102363071,Quant Active Growth Direct Plan,buy,22.671,441.0636,450.8031,-9999.50
4,2022-08-11,5102363071,Quant Infrastructure Growth Direct Plan,buy,432.598,23.1150,23.7855,-9999.50


#### Create the  function that calculates Xirr ( funciton copied from https://github.com/SunilVeeravalli/XIRR_in_Python/blob/main/xirr.py )

In [13]:
min_rate = 0
max_rate = 0
def npv(seq_of_rates: np.array, data: pd.DataFrame) -> tuple:
    global min_rate
    global max_rate
    
    for a_rate in seq_of_rates:
        max_date = data['Date'].max()
        data['npv'] = data['Amount'] * ((1 + (a_rate / 100)) ** ((max_date - data['Date']).dt.days / 365))
        
        if data['Amount'].sum() > 0:
            if data['npv'].sum() > 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
        else:
            if data['npv'].sum() < 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
    
    return min_rate, max_rate


def xirr(data: pd.DataFrame) -> float:
    # Finding out which column contains Date and Amount
    try:
        data.iloc[:, 0].astype(float)
        col_names = ['Amount', 'Date']
    except Exception as _:
        col_names = ['Date', 'Amount']
    
    # Renaming the columns accordingly and converting to correct data types
    data.columns = col_names
    data = data.assign(Date = pd.to_datetime(arg = data['Date'], infer_datetime_format = True, dayfirst = True).dt.date,
                       Amount = data['Amount'].astype(float))
    
    if data['Amount'].sum() > 0:
        step_values = [100, 10, 1, 0.01, 0.001, 0.0001]
        stop = 10000
    else:
        step_values = [-100, -10, -1, -0.01, -0.001, -0.0001]
        stop = -10000
    
    start = 0
    for i in range(len(step_values)):
        seq_of_rates = np.arange(start = start, stop = stop, step = step_values[i])
        start, stop = npv(seq_of_rates, data)
    
    return (start + stop) / 2


In [14]:
solution = []
mf= []
for i in range(0,len(mf_name)):
    mf = df[df['Name of the Fund'] == mf_name[i]]
    mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }), 
                   ignore_index = True, sort = True)
    ready_df = mf[['Date','Amount (INR)']]
    solution.append(xirr(ready_df))

In [15]:
final_df = pd.DataFrame(list(zip(mf_name.tolist(),solution)))

In [16]:
final_df.columns = ['Name of the Fund', 'Xirr']

### See the final performance of the mutual fund sorted based on the overall Xirr

In [17]:
final_df.sort_values('Xirr', ascending = False)

,Name of the Fund,Xirr
6,Quant Tax Growth Direct Plan,89.96645
7,Mirae Asset Tax Saver Growth Direct Plan,48.24815
13,ICICI Prudential Infrastructure Growth Direct ...,31.50595
5,Canara Robeco Small Cap Growth Direct Plan,24.94505
2,Quant Infrastructure Growth Direct Plan,20.95415
4,PGIM India Midcap Opportunities Growth Direct ...,16.17075
3,Quant Active Growth Direct Plan,14.12595
1,Quant Mid Cap Growth Direct Plan,13.64975
8,Axis Small Cap Growth Direct Plan,12.86595
15,Axis Growth Opportunities Growth Direct Plan,6.97595


In [18]:
final_df.sort_values('Name of the Fund', ascending = True)

,Name of the Fund,Xirr
15,Axis Growth Opportunities Growth Direct Plan,6.97595
8,Axis Small Cap Growth Direct Plan,12.86595
5,Canara Robeco Small Cap Growth Direct Plan,24.94505
10,ICICI Prudential Commodities Growth Direct Plan,5.00745
13,ICICI Prudential Infrastructure Growth Direct ...,31.50595
12,ICICI Prudential Technology Growth Direct Plan,-24.55555
9,Mirae Asset Emerging Bluechip Growth Direct Plan,6.84555
7,Mirae Asset Tax Saver Growth Direct Plan,48.24815
14,PGIM India Flexi Cap Growth Direct Plan,-1.59775
4,PGIM India Midcap Opportunities Growth Direct ...,16.17075


In [19]:
final_df

,Name of the Fund,Xirr
0,Quant Small Cap Growth Direct Plan,-1.52415
1,Quant Mid Cap Growth Direct Plan,13.64975
2,Quant Infrastructure Growth Direct Plan,20.95415
3,Quant Active Growth Direct Plan,14.12595
4,PGIM India Midcap Opportunities Growth Direct ...,16.17075
5,Canara Robeco Small Cap Growth Direct Plan,24.94505
6,Quant Tax Growth Direct Plan,89.96645
7,Mirae Asset Tax Saver Growth Direct Plan,48.24815
8,Axis Small Cap Growth Direct Plan,12.86595
9,Mirae Asset Emerging Bluechip Growth Direct Plan,6.84555
